In [2]:
import requests
import pandas as pd
import numpy as np

In [4]:
url = 'http://127.0.0.1:25510/list/roots?sec=STOCK'
headers = {'Accept': 'application/json'}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    # The request was successful
    json_data = response.json()
    print(json_data)
else:
    # Request failed
    print(f"Request failed with status code {response.status_code}")
    print(response.text)

{'header': {'id': 1, 'latency_ms': 116, 'error_type': 'null', 'error_msg': 'null', 'next_page': 'null', 'format': None}, 'response': ['A', 'AA', 'AAA', 'AAAP', 'AAAU', 'AABA', 'AAC', 'AAC.U', 'AAC.WS', 'AACC', 'AACG', 'AACI', 'AACIU', 'AACIW', 'AACOU', 'AACOW', 'AACQ', 'AACQU', 'AACQW', 'AACT', 'AACT.U', 'AACT.WS', 'AADI', 'AADR', 'AAIC', 'AAIC.PR.B', 'AAIC.PR.C', 'AAIN', 'AAIR', 'AAIT', 'AAL', 'AALCP', 'AAM.PR.A', 'AAMC', 'AAME', 'AAN', 'AAOI', 'AAON', 'AAP', 'AAPB', 'AAPC', 'AAPD', 'AAPH', 'AAPL', 'AAPU', 'AAPY', 'AAQC', 'AAQC.U', 'AAQC.WS', 'AAT', 'AATC', 'AAU', 'AAVL', 'AAWW', 'AAXJ', 'AAXN', 'AB', 'ABAC', 'ABAT', 'ABAX', 'ABB', 'ABBB', 'ABBV', 'ABC', 'ABCB', 'ABCD', 'ABCL', 'ABCM', 'ABCO', 'ABCW', 'ABDC', 'ABEO', 'ABEOW', 'ABEQ', 'ABEV', 'ABFS', 'ABG', 'ABGB', 'ABGI', 'ABHD', 'ABIL', 'ABILW', 'ABIO', 'ABL', 'ABLLW', 'ABLV', 'ABLVW', 'ABLX', 'ABM', 'ABMD', 'ABNB', 'ABOS', 'ABR', 'ABR.PR.E', 'ABSI', 'ABST', 'ABT', 'ABTL', 'ABTX', 'ABUS', 'ABVA', 'ABVC', 'ABVX', 'ABY', 'AC', 'ACA', '

In [8]:
class ThetaDataAPI:
    def __init__(self):
        self.base_url = 'http://127.0.0.1:25510/'

    def _get_req(self, url: str, headers: dict, params: dict=None):
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Request failed with status code {response.status_code}")
            print(response.text)

    def get_roots(self, security_type: str='OPTION'):
        url = f'{self.base_url}list/roots?sec={security_type}'
        headers = {'Accept': 'application/json'}
        return self._get_req(url=url, headers=headers)

    def get_expirations(self, root:str):
        url = f'{self.base_url}list/expirations?root={root}'
        headers = {'Accept': 'application/json'}
        return self._get_req(url=url, headers=headers)
    
    def get_strikes(self, root: str, exp: str):
        url = f'{self.base_url}list/strikes?root={root}&exp={exp}'
        headers = {'Accept': 'application/json'}
        return self._get_req(url=url, headers=headers)
    
    def get_dates(self, root: str, security_type: str="option", exp: str=None, strike_right: tuple[str, str]=None):
        url = f'{self.base_url}list/dates/{security_type}/quote'
        url_params = {}
        url_params["root"] = root
        if exp is not None:
            url_params["exp"] = exp
        if strike_right is not None:
            url_params["strike"] = strike_right[0]
            url_params["right"] = strike_right[1]
        headers = {'Accept': 'application/json'}
        return self._get_req(url=url, headers=headers, params=url_params)   



In [6]:
api = ThetaDataAPI()
api.get_expirations(root='AAPL')

{'header': {'id': 2,
  'latency_ms': 25,
  'error_type': 'null',
  'error_msg': 'null',
  'next_page': 'null',
  'format': None},
 'response': [20120601,
  20120608,
  20120616,
  20120622,
  20120629,
  20120706,
  20120713,
  20120721,
  20120727,
  20120803,
  20120810,
  20120818,
  20120824,
  20120831,
  20120907,
  20120914,
  20120922,
  20120928,
  20121005,
  20121012,
  20121020,
  20121026,
  20121102,
  20121109,
  20121117,
  20121123,
  20121130,
  20121207,
  20121214,
  20121222,
  20121228,
  20130104,
  20130111,
  20130119,
  20130125,
  20130201,
  20130208,
  20130216,
  20130222,
  20130301,
  20130308,
  20130316,
  20130322,
  20130328,
  20130405,
  20130412,
  20130420,
  20130426,
  20130503,
  20130510,
  20130518,
  20130524,
  20130531,
  20130607,
  20130614,
  20130622,
  20130628,
  20130705,
  20130712,
  20130720,
  20130726,
  20130802,
  20130809,
  20130817,
  20130823,
  20130830,
  20130906,
  20130913,
  20130921,
  20130927,
  20131004,
  2013

In [9]:
client = ThetaDataAPI()
root = "AAPL"
expiries = client.get_expirations(root=root)["response"]
exp = expiries[len(expiries)-50]
print(expiries)
strikes = client.get_strikes(root=root, exp=exp)
print(strikes)
dates = client.get_dates(root=root, exp=exp)
print(dates)

[20120601, 20120608, 20120616, 20120622, 20120629, 20120706, 20120713, 20120721, 20120727, 20120803, 20120810, 20120818, 20120824, 20120831, 20120907, 20120914, 20120922, 20120928, 20121005, 20121012, 20121020, 20121026, 20121102, 20121109, 20121117, 20121123, 20121130, 20121207, 20121214, 20121222, 20121228, 20130104, 20130111, 20130119, 20130125, 20130201, 20130208, 20130216, 20130222, 20130301, 20130308, 20130316, 20130322, 20130328, 20130405, 20130412, 20130420, 20130426, 20130503, 20130510, 20130518, 20130524, 20130531, 20130607, 20130614, 20130622, 20130628, 20130705, 20130712, 20130720, 20130726, 20130802, 20130809, 20130817, 20130823, 20130830, 20130906, 20130913, 20130921, 20130927, 20131004, 20131011, 20131019, 20131025, 20131101, 20131108, 20131116, 20131122, 20131129, 20131206, 20131213, 20131221, 20131227, 20140103, 20140110, 20140118, 20140124, 20140131, 20140207, 20140214, 20140222, 20140228, 20140307, 20140314, 20140322, 20140328, 20140404, 20140411, 20140419, 20140425,

TypeError: ThetaDataAPI._get_req() got an unexpected keyword argument 'params'